<a href="https://colab.research.google.com/github/Juliana001/Utilizades-Computacionais/blob/main/JPGparaSVGemPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala dependências (executar no Colab apenas uma vez)
!pip install pymupdf opencv-python numpy cairosvg pillow

# Upload do arquivo PDF achatado
from google.colab import files
uploaded = files.upload()

# Define o caminho do arquivo de entrada (pegando o primeiro upload)
input_pdf = list(uploaded.keys())[0]
output_pdf = input_pdf.replace(".pdf", "_SVG.pdf")
print(f"Arquivo carregado: {input_pdf}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00


Saving EntreReflexo.pdf to EntreReflexo.pdf
Arquivo carregado: EntreReflexo.pdf


In [ ]:
import fitz  # PyMuPDF
import numpy as np
import cv2

# === Parâmetros para ajustar qualidade ===
DPI_RENDER = 300
CANNY_LOW = 80
CANNY_HIGH = 200
MIN_CONTOUR_AREA = 50
APPROX_EPS_FRAC = 0.0025
INVERT_COLORS_FOR_HIGHKEY = False

def has_vectors_or_text(page):
    try:
        if page.get_drawings(): return True
    except: pass
    try:
        raw = page.get_text("rawdict") or {}
        for b in raw.get("blocks", []):
            if "lines" in b:
                for line in b["lines"]:
                    if line.get("spans"):
                        return True
    except: pass
    return False

def page_to_image(page):
    zoom = DPI_RENDER / 72.0
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, colorspace=fitz.csGRAY, alpha=False)
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w)
    return img, zoom

def vectorize_image(img_gray):
    img = img_gray.copy()
    if INVERT_COLORS_FOR_HIGHKEY:
        img = 255 - img
    img = cv2.GaussianBlur(img, (3, 3), 0)
    _, th = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    edges = cv2.Canny(th, CANNY_LOW, CANNY_HIGH)
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=1)

    contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    simplified = []
    for cnt in contours:
        if cv2.contourArea(cnt) < MIN_CONTOUR_AREA:
            continue
        eps = APPROX_EPS_FRAC * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, eps, True)
        if len(approx) >= 3:
            simplified.append(approx.squeeze(axis=1))
    return simplified

def contours_to_svg(contours, width, height):
    def path_d(pts):
        if pts.ndim != 2 or pts.shape[0] < 3:
            return ""
        cmds = [f"M {pts[0,0]} {pts[0,1]}"]
        for i in range(1, pts.shape[0]):
            x, y = pts[i]
            cmds.append(f"L {x} {y}")
        cmds.append("Z")
        return " ".join(cmds)

    paths = []
    for c in contours:
        d = path_d(c)
        if d:
            paths.append(f'<path d="{d}" fill="none" stroke="black" stroke-width="1"/>')

    return f'''<svg xmlns="http://www.w3.org/2000/svg"
     width="{width}" height="{height}" viewBox="0 0 {width} {height}">
  {"".join(paths)}
</svg>'''

# === Processamento do PDF ===
src = fitz.open(input_pdf)
out = fitz.open()

for i, page in enumerate(src):
    print(f"[{i+1}/{len(src)}] Processando página...")
    width_pt, height_pt = page.rect.width, page.rect.height

    if has_vectors_or_text(page):
        out.insert_pdf(src, from_page=i, to_page=i)
        print("  - Já tem vetores/texto → copiado sem alterar.")
        continue

    img, zoom = page_to_image(page)
    contours = vectorize_image(img)

    if not contours:
        print("  - Sem contornos úteis → mantendo imagem raster.")
        p = out.new_page(width=width_pt, height=height_pt)
        mat = fitz.Matrix(DPI_RENDER/72, DPI_RENDER/72)
        pix = page.get_pixmap(matrix=mat, colorspace=fitz.csRGB, alpha=False)
        p.insert_image(p.rect, pixmap=pix, keep_proportion=True)
        continue

    svg = contours_to_svg(contours, img.shape[1], img.shape[0])
    p = out.new_page(width=width_pt, height=height_pt)
    p.insert_svg(svg, rect=p.rect, keep_proportion=False)
    print(f"  - Vetorizado com {len(contours)} contornos.")

out.save(output_pdf, garbage=4, deflate=True, clean=True)
out.close()
src.close()

print(f"\n[OK] PDF final salvo em: {output_pdf}")


[1/1] Processando página...
  - Já tem vetores/texto → copiado sem alterar.

[OK] PDF final salvo em: EntreReflexo_corel.pdf


In [ ]:
import sys
import io
import math
import fitz  # PyMuPDF
import numpy as np
import cv2
from typing import Tuple, List

# -------------------------
# Parâmetros ajustáveis
# -------------------------
DPI_RENDER = 300
CANNY_LOW = 80
CANNY_HIGH = 200
MIN_CONTOUR_AREA = 50        # pixels (área mínima para aceitar um contorno)
APPROX_EPS_FRAC = 0.0025     # fração do perímetro para simplificar contornos
INVERT_COLORS_FOR_HIGHKEY = False  # ligue se o original for claro em fundo escuro

def has_vectors_or_text(page: fitz.Page) -> bool:
    """Retorna True se a página já tiver vetores (desenhos) ou texto."""
    try:
        if page.get_drawings():  # objetos vetoriais (paths)
            return True
    except Exception:
        pass
    try:
        raw = page.get_text("rawdict") or {}
        blocks = raw.get("blocks", [])
        for b in blocks:
            if "lines" in b:  # presença de linhas de texto
                for line in b["lines"]:
                    if line.get("spans"):
                        return True
    except Exception:
        pass
    return False

def page_to_image(page: fitz.Page, dpi: int = DPI_RENDER) -> Tuple[np.ndarray, float]:
    """Renderiza a página em imagem (grayscale) e retorna (img, scale).
    scale = pixels_por_ponto (PDF usa pontos, 72 pt = 1 in)."""
    zoom = dpi / 72.0
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, colorspace=fitz.csGRAY, alpha=False)
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w)
    return img, zoom

def vectorize_image(img_gray: np.ndarray) -> List[np.ndarray]:
    """Extrai contornos vetoriais a partir de uma imagem em tons de cinza."""
    img = img_gray.copy()
    if INVERT_COLORS_FOR_HIGHKEY:
        img = 255 - img

    # Normalização e redução de ruído leve
    img = cv2.GaussianBlur(img, (3, 3), 0)

    # Binarização automática (Otsu) para separar figura/fundo
    _, th = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Aumenta contraste de bordas
    edges = cv2.Canny(th, CANNY_LOW, CANNY_HIGH)

    # Fecha pequenas lacunas
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Encontra contornos externos (sem buracos) e internos
    contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    simplified = []
    for cnt in contours:
        if cv2.contourArea(cnt) < MIN_CONTOUR_AREA:
            continue
        # simplifica o contorno (menos pontos → SVG menor)
        eps = APPROX_EPS_FRAC * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, eps, True)
        if len(approx) >= 3:
            simplified.append(approx.squeeze(axis=1))  # (N,2)
    return simplified

def contours_to_svg(contours: List[np.ndarray], width: int, height: int) -> str:
    """Converte contornos em paths SVG. Coordenadas em pixels; y invertido para SVG (origem no topo)."""
    def path_d(pts: np.ndarray) -> str:
        if pts.ndim != 2 or pts.shape[0] < 3:
            return ""
        # MoveTo/Lineto; fecha em Z
        cmds = [f"M {pts[0,0]} {pts[0,1]}"]
        for i in range(1, pts.shape[0]):
            x, y = pts[i]
            cmds.append(f"L {x} {y}")
        cmds.append("Z")
        return " ".join(cmds)

    # SVG com viewBox em pixels; sem fill/stroke explícitos (Corel ajusta)
    paths = []
    for c in contours:
        d = path_d(c)
        if d:
            paths.append(f'<path d="{d}" fill="none" stroke="black" stroke-width="1"/>')

    svg = f'''<?xml version="1.0" standalone="no"?>
<svg xmlns="http://www.w3.org/2000/svg"
     width="{width}" height="{height}" viewBox="0 0 {width} {height}">
  {"".join(paths)}
</svg>'''
    return svg

def insert_svg_on_new_page(out_doc: fitz.Document, width_pt: float, height_pt: float, svg_text: str):
    """Cria uma página no PDF de saída e insere o SVG ocupando a página toda."""
    page = out_doc.new_page(width=width_pt, height=height_pt)
    # Inserção na caixa da página toda
    page.insert_svg(svg_text, rect=page.rect, keep_proportion=False)

def main(input_pdf: str, output_pdf: str):
    src = fitz.open(input_pdf)
    out = fitz.open()

    for i, page in enumerate(src):
        print(f"[{i+1}/{len(src)}] Processando página...")
        # Tamanho da página em pontos (1/72 inch)
        width_pt, height_pt = page.rect.width, page.rect.height

        if has_vectors_or_text(page):
            # Copia a página original (mantém vetores/texto)
            out.insert_pdf(src, from_page=i, to_page=i)
            print("  - Página já vetorial/textual → copiada sem alterar.")
            continue

        # Caso seja básicamente imagem → vetorizar
        img, zoom = page_to_image(page, dpi=DPI_RENDER)
        h, w = img.shape[:2]

        contours = vectorize_image(img)
        if not contours:
            # Se não achou contorno significativo, cai no plano B: insere a imagem como bitmap
            # (assim pelo menos abre no Corel; você pode vetorizar manualmente depois)
            print("  - Sem contornos úteis; inserindo imagem raster (fallback).")
            p = out.new_page(width=width_pt, height=height_pt)
            # Converte numpy para pixmap e insere
            # Renderiza novamente em RGB para qualidade
            mat = fitz.Matrix(DPI_RENDER/72, DPI_RENDER/72)
            pix = page.get_pixmap(matrix=mat, colorspace=fitz.csRGB, alpha=False)
            img_rect = p.rect
            p.insert_image(img_rect, pixmap=pix, keep_proportion=True)
            continue

        # Gera SVG em coordenadas de pixels
        svg = contours_to_svg(contours, w, h)

        # Inserir SVG ocupando a página inteira
        insert_svg_on_new_page(out, width_pt, height_pt, svg)
        print(f"  - Vetorizado: {len(contours)} contornos inseridos como SVG.")

    # Salvar o PDF final
    out.save(output_pdf, garbage=4, deflate=True, clean=True)
    out.close()
    src.close()
    print(f"\n[OK] PDF salvo em: {output_pdf}")

if __name__ == "__main__":
    if len(sys.argv) < 3:
        print("Uso: python vetoriza_pdf_para_corel.py entrada.pdf saida_corel.pdf")
        sys.exit(1)
    main(sys.argv[1], sys.argv[2])


[INFO] O PDF tem 1 páginas.
[OK] Página 1 exportada para SVG.

[FINALIZADO] Arquivos SVG gerados em 'saida'
[FINALIZADO] Novo PDF 'arquivo_corel.pdf' criado (lista os SVGs).
